# Phages from Australia

What are the unique and wonderful phage from Australia that we should be studying?

In [1]:
# A lot of this is not used, but we import it so we have it later!
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import seaborn as sns
import numpy as np

import math
import re

from PhiSpyAnalysis import theils_u, DateConverter, printmd
from PhiSpyAnalysis import read_phages, read_gtdb, read_checkv, read_base_pp, read_categories, read_metadata, read_gbk_metadata

from scipy.stats import pearsonr, f_oneway
from sklearn.linear_model import LinearRegression
from sklearn import decomposition
from sklearn.ensemble import RandomForestClassifier

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, tukeyhsd, MultiComparison
from statsmodels.multivariate.manova import MANOVA


import subprocess
import gzip


In [2]:
rv = re.compile('^\\w+')
def remove_ver(x):
    return rv.search(str(x)).group()

In [3]:
# GTDB
gtdb = read_gtdb()
gtdb['assembly_nover'] = gtdb['assembly_accession'].apply(remove_ver)
gtdb[['assembly_accession', 'assembly_nover']]

,assembly_accession,assembly_nover
0,GCA_000006155.2,GCA_000006155
1,GCA_000007385.1,GCA_000007385
2,GCA_000008605.1,GCA_000008605
3,GCA_000010565.1,GCA_000010565
4,GCA_000013845.2,GCA_000013845
...,...,...
191522,GCA_902166935.1,GCA_902166935
191523,GCA_902166945.1,GCA_902166945
191524,GCA_902167295.1,GCA_902167295
191525,GCA_902167305.1,GCA_902167305


In [4]:
# RAST
# the full data set. Don't try this at home!
# metadf = pd.read_csv("../small_data/patric_genome_metadata.tsv.gz", compression='gzip', header=0, delimiter="\t")
rast = read_metadata()
rast['assembly_nover'] = rast['assembly_accession'].apply(remove_ver)
rast[['assembly_accession', 'assembly_nover']]

,assembly_accession,assembly_nover
0,GCA_000003135.1,GCA_000003135
1,GCA_000003215.1,GCA_000003215
2,GCA_000003645.1,GCA_000003645
3,GCA_000003925.1,GCA_000003925
4,GCA_000003955.1,GCA_000003955
...,...,...
320171,GCF_900167595.1,GCF_900167595
320172,GCF_900167605.1,GCF_900167605
320173,GCF_900167615.1,GCF_900167615
320174,GCF_900167625.1,GCF_900167625


In [5]:
# GenBank
gbk = read_gbk_metadata()
gbk['assembly_nover'] = gbk['assembly_accession'].apply(remove_ver)
gbk[['assembly_accession', 'assembly_nover']]

c:\workspace\edwa0468\githubs\phispyanalysis\venv\lib\site-packages\IPython\core\interactiveshell.py:3361: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,assembly_accession,assembly_nover
0,GCA_900128725.1,GCA_900128725
1,GCA_008244535.1,GCA_008244535
2,GCA_011046815.1,GCA_011046815
3,GCA_011054035.1,GCA_011054035
4,GCA_011331185.1,GCA_011331185
...,...,...
949928,GCA_017746755.1,GCA_017746755
949929,GCA_017746715.1,GCA_017746715
949930,GCA_013266695.1,GCA_013266695
949931,GCA_015831295.1,GCA_015831295


In [6]:
phagesdf = read_phages(maxcontigs=-1) # this disables contig length filtering
phagesdf['assembly_nover'] = phagesdf['assembly_accession'].apply(remove_ver)
phagesdf

Please note that this was run with git commit e73598a that has 567,404 genomes parsed.
Initially there were 3,265,453 kept phages, but after filtering we kept 3,265,453 prophages from 567,399 genomes

,assembly_accession,assembly_name,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage,assembly_nover
0,GCA_000043285.1,ASM4328v1,705557.0,1.0,1.0,2.0,0.0,2.0,0.0,0.0,GCA_000043285
1,GCA_000046685.1,ASM4668v1,1581384.0,1.0,1.0,18.0,3.0,0.0,15.0,97011.0,GCA_000046685
2,GCA_000046705.1,ASM4670v1,1931047.0,1.0,1.0,36.0,7.0,0.0,29.0,278490.0,GCA_000046705
3,GCA_000046845.1,ASM4684v1,3598621.0,1.0,1.0,13.0,3.0,4.0,6.0,83112.0,GCA_000046845
4,GCA_000047365.1,ASM4736v1,4840898.0,3.0,3.0,22.0,3.0,3.0,16.0,213427.0,GCA_000047365
...,...,...,...,...,...,...,...,...,...,...,...
567399,GCA_905187425.1,Xanthomonas_sp._CPBF_424_-_hybrid_assembly,4900930.0,1.0,1.0,46.0,7.0,9.0,30.0,214278.0,GCA_905187425
567400,GCA_905188235.1,ASM90518823v1,4963609.0,1.0,1.0,29.0,4.0,4.0,21.0,178993.0,GCA_905188235
567401,GCA_905219375.1,QI0054,3034314.0,1.0,1.0,12.0,4.0,3.0,5.0,178527.0,GCA_905219375
567402,GCA_905219385.1,QI0055,3034113.0,1.0,1.0,12.0,4.0,3.0,5.0,178526.0,GCA_905219385


In [7]:
rast.columns

Index(['assembly_accession', 'genome_id', 'genome_name', 'organism_name',
       'taxon_id', 'genome_status', 'strain', 'serovar', 'biovar', 'pathovar',
       'mlst', 'other_typing', 'culture_collection', 'type_strain',
       'completion_date', 'publication', 'bioproject_accession',
       'biosample_accession', 'genbank_accessions', 'refseq_accessions',
       'sequencing_centers', 'sequencing_status', 'sequencing_platform',
       'sequencing_depth', 'assembly_method', 'chromosomes', 'plasmids',
       'contigs', 'sequences', 'genome_length', 'gc_content', 'patric_cds',
       'brc1_cds', 'refseq_cds', 'isolation_site', 'isolation_source',
       'isolation_comments', 'collection_date', 'isolation_country',
       'geographic_location', 'latitude', 'longitude', 'altitude', 'depth',
       'other_environmental', 'host_name', 'host_gender', 'host_age',
       'host_health', 'body_sample_site', 'body_sample_subsite',
       'other_clinical', 'antimicrobial_resistance',
       'antimic

# Kangaroos and Koalas

Here, we limit the data to bacteria from kangaroos and koalas to see if there are some interesting things. 

Unfortunately, it seems they are mostly Proteobacteria (yawn) 

In [8]:
kk = rast[(rast['host_name'].str.contains('koala', case=False, na=False)) | (rast['host_name'].str.contains('kangaroo', case=False, na=False))]
kk

,assembly_accession,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,...,temperature_range,optimal_temperature,salinity,oxygen_requirement,habitat,disease,comments,additional_metadata,isolation_date,assembly_nover
528,GCA_000017105.1,4.349225e+05,Coxiella burnetii Dugway 5J108-111,Coxiella burnetii Dugway 5J108-111,434922,Complete,Dugway 5J108-111,NaN,NaN,NaN,...,Mesophilic,37,NaN,Facultative,Host-associated,Q fever,Coxiella burnetii Dugway 5J108-111. Coxiella b...,NaN,NaN,GCA_000017105
8863,GCA_000341355.1,1.094489e+06,Bartonella australis Aust/NH1,Bartonella australis Aust/NH1,1094489,Complete,Aust/NH1,NaN,NaN,NaN,...,Unknown,NaN,Unknown,Unknown,HostAssociated,NaN,Whole-genome sequencing of Bartonella australi...,NaN,1999.683094,GCA_000341355
24579,GCA_000689835.1,1.331229e+06,Bordetella bronchiseptica MBORD681,NaN,1331229,WGS,MBORD681,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,In order to understand the speciation and evol...,NaN,NaN,GCA_000689835
24580,GCA_000689855.1,1.331230e+06,Bordetella bronchiseptica MBORD698,NaN,1331230,WGS,MBORD698,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,In order to understand the speciation and evol...,NaN,NaN,GCA_000689855
37899,GCA_001689645.1,1.807743e+06,Bacteroidales bacterium K1,NaN,1807743,WGS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The Bacteroidales group S24-7 is recognized as...,sample_type:metagenomic assembly,2015.017796,GCA_001689645
37900,GCA_001689655.1,1.807744e+06,Bacteroidales bacterium K10,NaN,1807744,WGS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The Bacteroidales group S24-7 is recognized as...,sample_type:metagenomic assembly,2015.017796,GCA_001689655
38407,GCA_001729965.1,1.561964e+06,Methanosphaera sp. WGK6,NaN,1561964,WGS,WGK6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Aim to isolate and sequence methanogens origin...,NaN,2008.685147,GCA_001729965
42914,GCA_001941025.1,1.930016e+06,Lachnospiraceae bacterium Zagget3,NaN,1930016,WGS,Zagget3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The study goal was to identify the top 15 popu...,sample_type:Population genome extracted from m...,2011.316222,GCA_001941025
42915,GCA_001941045.1,1.930015e+06,Lachnospiraceae bacterium Zagget2,NaN,1930015,WGS,Zagget2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The study goal was to identify the top 15 popu...,sample_type:Population genome extracted from m...,2011.316222,GCA_001941045
42916,GCA_001941055.1,1.930001e+06,Ruminococcus sp. Zagget7,NaN,1930001,WGS,Zagget7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,The study goal was to identify the top 15 popu...,sample_type:Population genome extracted from m...,2011.316222,GCA_001941055


In [9]:
acccol = 'assembly_accession'

,assembly_accession,contig_count,genome_size,Contigs,Genome length,same


### Merge the tables together

First we merge the gtdb taxonomy with the RAST data just for kangaroos and koalas, and then we merge with the number of prophages.

In [16]:
kk_tax = pd.merge(kk,
                  gtdb[[acccol, 'gtdb_taxonomy', 'domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']],
                  how='inner', left_on=acccol, right_on=acccol)
kk_tax.head()

,assembly_accession,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,...,isolation_date,assembly_nover,gtdb_taxonomy,domain,phylum,class,order,family,genus,species
0,GCA_000017105.1,434922.5,Coxiella burnetii Dugway 5J108-111,Coxiella burnetii Dugway 5J108-111,434922,Complete,Dugway 5J108-111,NaN,NaN,NaN,...,NaN,GCA_000017105,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Coxiellales,Coxiellaceae,Coxiella,Coxiella burnetii
1,GCA_000341355.1,1094489.3,Bartonella australis Aust/NH1,Bartonella australis Aust/NH1,1094489,Complete,Aust/NH1,NaN,NaN,NaN,...,1999.683094,GCA_000341355,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Rhizobiaceae,Bartonella,Bartonella australis
2,GCA_000689835.1,1331229.3,Bordetella bronchiseptica MBORD681,NaN,1331229,WGS,MBORD681,NaN,NaN,NaN,...,NaN,GCA_000689835,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Bordetella,Bordetella pertussis
3,GCA_000689855.1,1331230.3,Bordetella bronchiseptica MBORD698,NaN,1331230,WGS,MBORD698,NaN,NaN,NaN,...,NaN,GCA_000689855,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,Bacteria,Proteobacteria,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Bordetella,Bordetella pertussis
4,GCA_001689645.1,1807743.4,Bacteroidales bacterium K1,NaN,1807743,WGS,NaN,NaN,NaN,NaN,...,2015.017796,GCA_001689645,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Muribaculaceae,CAG-873,CAG-873 sp002633115


In [29]:
kk_tp = pd.merge(kk_tax,
                  phagesdf,
                  how='left', left_on=acccol, right_on=acccol)
kk_tp

,assembly_accession,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,...,assembly_name,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage,assembly_nover_y
0,GCA_000017105.1,4.349225e+05,Coxiella burnetii Dugway 5J108-111,Coxiella burnetii Dugway 5J108-111,434922,Complete,Dugway 5J108-111,NaN,NaN,NaN,...,ASM1710v1,2212937.0,2.0,2.0,10.0,0.0,6.0,4.0,0.0,GCA_000017105
1,GCA_000341355.1,1.094489e+06,Bartonella australis Aust/NH1,Bartonella australis Aust/NH1,1094489,Complete,Aust/NH1,NaN,NaN,NaN,...,ASM34135v1,1596490.0,1.0,1.0,8.0,3.0,0.0,5.0,79039.0,GCA_000341355
2,GCA_000689835.1,1.331229e+06,Bordetella bronchiseptica MBORD681,NaN,1331229,WGS,MBORD681,NaN,NaN,NaN,...,gbb33v01,5158310.0,70.0,34.0,58.0,5.0,14.0,39.0,126457.0,GCA_000689835
3,GCA_000689855.1,1.331230e+06,Bordetella bronchiseptica MBORD698,NaN,1331230,WGS,MBORD698,NaN,NaN,NaN,...,gbb34v01,5151735.0,52.0,39.0,55.0,5.0,11.0,39.0,111816.0,GCA_000689855
4,GCA_001689645.1,1.807743e+06,Bacteroidales bacterium K1,NaN,1807743,WGS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GCA_001689655.1,1.807744e+06,Bacteroidales bacterium K10,NaN,1807744,WGS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GCA_001941025.1,1.930016e+06,Lachnospiraceae bacterium Zagget3,NaN,1930016,WGS,Zagget3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,GCA_001941045.1,1.930015e+06,Lachnospiraceae bacterium Zagget2,NaN,1930015,WGS,Zagget2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GCA_001941065.1,1.930002e+06,Alistipes sp. Zagget8,NaN,1930002,WGS,Zagget8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,GCA_001941075.1,1.930019e+06,Oscillospiraceae bacterium Zagget6,NaN,1930019,WGS,Zagget6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## NOTE: We don't have prophage predictions for all Australian isolates!

We only have pp predictions for 10 isolates, and so for some of these kept is NaN

In [19]:
kk_tp.shape

(10, 86)

In [38]:
kk_tp[['assembly_accession', 'genome_name', 'host_name', 'isolation_country', 'phylum', 'species', 'Kept']].sort_values('phylum')

,assembly_accession,genome_name,host_name,isolation_country,phylum,species,Kept
20,GCA_004155475.1,Bifidobacterium pseudolongum subsp. globosum s...,Kangaroo,Italy,Actinobacteriota,Bifidobacterium globosum,1.0
21,GCA_004155625.1,Bifidobacterium pseudolongum subsp. globosum s...,Kangaroo,NaN,Actinobacteriota,Bifidobacterium globosum,3.0
11,GCA_001941205.1,Synergistes sp. Zagget9,koala,Australia,Bacteroidota,Amulumruptor caecigallinarius,NaN
5,GCA_001689655.1,Bacteroidales bacterium K10,Koala,Australia,Bacteroidota,Amulumruptor caecigallinarius,NaN
8,GCA_001941065.1,Alistipes sp. Zagget8,koala,Australia,Bacteroidota,Alistipes sp001941065,NaN
18,GCA_002633305.1,Muribaculaceae bacterium Zag10,koala,Australia,Bacteroidota,Amulumruptor caecigallinarius,NaN
4,GCA_001689645.1,Bacteroidales bacterium K1,Koala,Australia,Bacteroidota,CAG-873 sp002633115,NaN
16,GCA_002633115.1,Muribaculaceae bacterium Zag1,koala,Australia,Bacteroidota,CAG-873 sp002633115,NaN
17,GCA_002633155.1,Candidatus Melainabacteria bacterium Zag15,koala,Australia,Cyanobacteria,Zag1 sp001917115,NaN
13,GCA_002097375.1,Staphylococcus aureus strain N13/1/662,kangaroo,Australia,Firmicutes,Staphylococcus aureus,4.0


# Phages from Australia

Repeat the above, but with isolation country = Australia

In [41]:
aus = rast[rast['isolation_country'].str.contains('australia', case=False, na=False)]
aus_tax = pd.merge(aus,
                  gtdb[[acccol, 'gtdb_taxonomy', 'domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']],
                  how='inner', left_on=acccol, right_on=acccol)
aus_tp = pd.merge(aus_tax,
                  phagesdf,
                  how='left', left_on=acccol, right_on=acccol)
aus_tp

,assembly_accession,genome_id,genome_name,organism_name,taxon_id,genome_status,strain,serovar,biovar,pathovar,...,assembly_name,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage,assembly_nover_y
0,GCA_000014905.1,2.342671e+05,Candidatus Solibacter usitatus Ellin6076,Candidatus Solibacter usitatus Ellin6076,234267,Complete,Ellin6076,NaN,NaN,NaN,...,ASM1490v1,9965640.0,1.0,1.0,60.0,8.0,11.0,41.0,202405.0,GCA_000014905
1,GCA_000016845.1,4.441783e+05,Brucella ovis ATCC 25840,Brucella ovis ATCC 25840,444178,Complete,ATCC25840,NaN,NaN,NaN,...,ASM1684v1,3275590.0,2.0,2.0,16.0,2.0,5.0,9.0,82310.0,GCA_000016845
2,GCA_000024005.1,4.859186e+05,Chitinophaga pinensis DSM 2588,Chitinophaga pinensis DSM 2588,485918,Complete,DSM 2588,NaN,NaN,NaN,...,ASM2400v1,9127347.0,1.0,1.0,23.0,1.0,0.0,22.0,52154.0,GCA_000024005
3,GCA_000026185.1,4.658172e+05,Erwinia tasmaniensis Et1/99,Erwinia tasmaniensis Et1/99,465817,Plasmid,Et1/99,NaN,NaN,NaN,...,ASM2618v1,4067864.0,6.0,6.0,34.0,4.0,6.0,24.0,144087.0,GCA_000026185
4,GCA_000143865.1,8.658673e+05,Mycoplasma mycoides subsp. mycoides SC str. Gl...,Mycoplasma mycoides subsp. mycoides SC str. Gl...,865867,Complete,Gladysdale,NaN,NaN,NaN,...,ASM14386v1,1193808.0,1.0,1.0,8.0,1.0,3.0,4.0,13721.0,GCA_000143865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,GCA_901212295.1,1.145171e+05,Aeromonas veronii bv. sobria strain BC88,NaN,114517,WGS,BC88,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2710,GCA_901482475.1,1.502107e+03,Clostridium perfringens strain NCTC8503,NaN,1502,WGS,NCTC8503,NaN,NaN,NaN,...,44738_A01,3574234.0,10.0,9.0,28.0,5.0,3.0,20.0,131311.0,GCA_901482475
2711,GCA_901482565.1,1.719267e+03,Corynebacterium pseudotuberculosis strain NCTC...,NaN,1719,Complete,NCTC4681,NaN,NaN,NaN,...,45076_G01,2337798.0,1.0,1.0,14.0,1.0,4.0,9.0,18350.0,GCA_901482565
2712,GCA_901687245.1,1.431451e+06,Helicobacter pylori PMSS1 strain Hp GFP,NaN,1431450,WGS,Hp GFP,NaN,NaN,NaN,...,25455_8_248,1602364.0,40.0,18.0,7.0,0.0,3.0,4.0,0.0,GCA_901687245


In [44]:
list(pd.unique(aus_tp['phylum']))

['Acidobacteriota',
 'Proteobacteria',
 'Bacteroidota',
 'Firmicutes',
 'Planctomycetota',
 'Verrucomicrobiota',
 'Cyanobacteria',
 'Firmicutes_E',
 'Actinobacteriota',
 'Spirochaetota',
 'Campylobacterota',
 'Desulfobacterota',
 'Firmicutes_A',
 'Thermotogota',
 'Firmicutes_B',
 'Chrysiogenetota',
 'Verrucomicrobiota_A',
 'Fibrobacterota',
 'Dependentiae',
 'Cloacimonadota',
 'Chloroflexota',
 'Deferribacterota',
 'Synergistota',
 'Fusobacteriota',
 'Firmicutes_C',
 'Patescibacteria',
 'Bdellovibrionota',
 'Armatimonadota',
 'Bdellovibrionota_C',
 'Myxococcota',
 'Omnitrophota',
 'UBP18',
 'Firmicutes_D',
 'Bipolaricaulota',
 'SAR324',
 'Methylomirabilota',
 'Zixibacteria']

# Ochre

What is happening in the Ochre world?

This set of bactereial orders comes from this paper: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0185252#sec008

```
Lenehan CE, Tobe SS, Smith RJ, Popelka-Filcoff RS. 2017. Microbial composition analyses by 16S rRNA sequencing: A proof of concept approach to provenance determination of archaeological ochre. PLoS One 12:e0185252.
```

I extracted the orders from `Table S2 - Order level relative proportion of matches to the Greengenes (13_08) database` by:
1. convert pdf to text: `pdftotext -layout pone.0185252.s003.pdf`
2. print the list of orders that end `ales`:  `perl -ne 'if (/(\S+ales)/) {print "$1\n"}' pone.0185252.s003.txt | sort -u | perl -ne 'chomp; print "\"$_\", "'`

Note: Evan kindly converted that file into [an excel file](Australia/Lenehan_Table_S2.xlsx) and we read that below



In [45]:
ochre_orders = ["Acidimicrobiales", "Actinomycetales", "Aeromonadales", "Alteromonadales", "Ardenscatenales", "Armatimonadales", "Bacillales", "Bacteroidales", "Burkholderiales", "Caldilineales", "Campylobacterales", "Caulobacterales", "Chloroflexales", "Chromatiales", "Chroococcales", "Chthoniobacterales", "Clostridiales", "Coriobacteriales", "Cytophagales", "Deinococcales", "Enterobacteriales", "Euzebyales", "Flavobacteriales", "Gaiellales", "Gemellales", "Gemmatales", "Gemmatimonadales", "Herpetosiphonales", "Lactobacillales", "Micrococcales", "Myxococcales", "Neisseriales", "Nitrosomonadales", "Nitrospirales", "Nostocales", "Pasteurellales", "Pirellulales", "Planctomycetales", "Pseudomonadales", "Rhizobiales", "Rhodobacterales", "Rhodospirillales", "Rhodothermales", "Rickettsiales", "Roseiflexales", "Rubrobacterales", "Saprospirales", "Solirubrobacterales", "Sphingobacteriales", "Sphingomonadales", "Spirobacillales", "Synergistales", "Syntrophobacterales", "Thermobaculales", "Thiotrichales", "Vibrionales", "Xanthomonadales"]

In [56]:
ochre_gtdb = gtdb[gtdb['order'].isin(ochre_orders)]
ochre_phage = pd.merge(ochre_gtdb,
                  phagesdf,
                  how='inner', left_on=acccol, right_on=acccol)
ochre_phage

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,assembly_name,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage,assembly_nover_y
0,GB_GCA_000006155.2,1916,93.12,0.00,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,...,ASM615v2,5370060.0,3.0,3.0,5.0,0.0,2.0,3.0,0.0,GCA_000006155
1,GB_GCA_000007385.1,0,99.82,0.00,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,...,ASM738v1,4941439.0,1.0,1.0,29.0,4.0,12.0,13.0,129141.0,GCA_000007385
2,GB_GCA_000013845.2,0,100.00,0.00,332,o__Clostridiales (UID1375),124,0.0,2428396,82.037966,...,ASM1384v2,2960088.0,4.0,4.0,12.0,2.0,4.0,6.0,56955.0,GCA_000013845
3,GB_GCA_000014305.1,0,97.38,0.00,475,o__Lactobacillales (UID544),267,0.0,1810429,86.362697,...,ASM1430v1,2096309.0,1.0,1.0,19.0,4.0,9.0,6.0,84555.0,GCA_000014305
4,GB_GCA_000014325.1,0,97.87,0.00,475,o__Lactobacillales (UID544),267,0.0,1808181,86.280609,...,ASM1432v1,2095698.0,1.0,1.0,23.0,4.0,14.0,5.0,74165.0,GCA_000014325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56758,RS_GCF_901971775.1,19,99.18,0.00,336,o__Lactobacillales (UID355),184,0.0,1864126,85.804187,...,INIA_P508,2172535.0,67.0,42.0,18.0,0.0,7.0,11.0,0.0,GCA_901971775
56759,RS_GCF_901971785.1,27,99.46,0.27,586,g__Lactobacillus (UID436),184,100.0,2781164,85.330494,...,INIA_P344,3259285.0,88.0,45.0,20.0,2.0,11.0,7.0,41804.0,GCA_901971785
56760,RS_GCF_901971795.1,29,99.46,1.27,586,g__Lactobacillus (UID436),184,100.0,2612902,85.095735,...,INIA_P540,3070544.0,80.0,51.0,12.0,2.0,4.0,6.0,57146.0,GCA_901971795
56761,RS_GCF_902141825.1,0,99.27,1.34,507,c__Gammaproteobacteria (UID4444),232,0.0,5117266,84.091758,...,CECT_9533,6085336.0,49.0,20.0,15.0,0.0,7.0,8.0,0.0,GCA_902141825


### Read the Lenehan et al results

This table has the results from Claire and Rachel's paper where they describe the microbiota of ochre using 16S. Evan converted it to an excel file, and I added average and sum columns to allow sorting.

In [63]:
xldata = pd.read_excel('Australia/Lenehan_Table_S2.xlsx', "Sortable")
xldata

,Taxon,Bookarto R1,Bookarto R2,Bookarto R3,Karrku R1,Karrku R2,Moana R1,Moana R2,Moana R3,Wilgie Mia R1,Wilgia Mia R2,Wilgia Mia R3,Average,Sum
0,Actinomycetales,0.304289,0.334295,0.382574,0.083093,0.417965,0.125024,0.152529,0.255049,0.070013,0.164839,0.046932,0.212418,2.336603
1,Bacillales,0.417965,0.239084,0.283131,0.268898,0.028082,0.099635,0.038276,0.037507,0.108867,0.008655,0.019042,0.140831,1.549144
2,Rhodospirillales,0.015388,0.020196,0.018465,0.046740,0.075784,0.000962,0.006924,0.013272,0.193691,0.472014,0.408155,0.115599,1.271590
3,Rhizobiales,0.027890,0.086747,0.085593,0.031737,0.095403,0.109636,0.043854,0.006732,0.278515,0.173302,0.216772,0.105108,1.156183
4,Rhodobacterales,0.020004,0.065974,0.033276,0.001346,0.060781,0.149452,0.043278,0.358723,0.001923,0.000385,0.100211,0.075941,0.835353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Sva0725,0.000000,0.000192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000192
97,Syntrophobacterales,0.000000,0.000192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000192
98,Thiotrichales,0.000000,0.000192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000192
99,TK10 (unknown order),0.000000,0.000192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000017,0.000192


In [83]:
xlorders = [x.strip() for x in xldata[xldata['Average'] > 0.01].iloc[:,0]]
xlgtdb = gtdb[gtdb['order'].isin(xlorders)]
pd.unique(xlgtdb['phylum'])

array(['Firmicutes', 'Proteobacteria', 'Actinobacteriota', 'Bacteroidota'],
      dtype=object)

In [76]:
gtdb

,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,trna_count,trna_selenocysteine_count,domain,phylum,class,order,family,genus,species,assembly_nover
0,GB_GCA_000006155.2,1916,93.12,0.00,1171,g__Bacillus (UID902),324,0.0,4305660,80.178992,...,31,0,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae_G,Bacillus_A,Bacillus_A anthracis,GCA_000006155
1,GB_GCA_000007385.1,0,99.82,0.00,481,c__Gammaproteobacteria (UID4202),276,0.0,4190634,84.805944,...,53,0,Bacteria,Proteobacteria,Gammaproteobacteria,Xanthomonadales,Xanthomonadaceae,Xanthomonas,Xanthomonas oryzae,GCA_000007385
2,GB_GCA_000008605.1,67,100.00,0.00,235,f__Spirochaetaceae (UID2512),124,0.0,1048744,92.155875,...,45,0,Bacteria,Spirochaetota,Spirochaetia,Treponematales,Treponemataceae,Treponema,Treponema pallidum,GCA_000008605
3,GB_GCA_000010565.1,0,100.00,0.63,295,p__Firmicutes (UID1022),158,0.0,2608397,86.217312,...,51,1,Bacteria,Firmicutes_B,Desulfotomaculia,Desulfotomaculales,Pelotomaculaceae,Pelotomaculum,Pelotomaculum thermopropionicum,GCA_000010565
4,GB_GCA_000013845.2,0,100.00,0.00,332,o__Clostridiales (UID1375),124,0.0,2428396,82.037966,...,95,1,Bacteria,Firmicutes_A,Clostridia,Clostridiales,Clostridiaceae,Clostridium_P,Clostridium_P perfringens,GCA_000013845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191522,RS_GCF_902166935.1,0,99.62,0.09,1312,g__Klebsiella (UID5140),336,0.0,4894244,87.293148,...,87,1,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae,GCA_902166935
191523,RS_GCF_902166945.1,0,99.94,0.72,1162,f__Enterobacteriaceae (UID5121),336,0.0,5162721,86.540042,...,86,1,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae,GCA_902166945
191524,RS_GCF_902167295.1,0,99.51,0.00,303,p__Bacteroidetes (UID2591),203,0.0,5737447,90.122088,...,58,0,Bacteria,Bacteroidota,Bacteroidia,Chitinophagales,Chitinophagaceae,Chitinophaga,Chitinophaga pinensis_A,GCA_902167295
191525,RS_GCF_902167305.1,0,99.51,0.00,303,p__Bacteroidetes (UID2591),203,0.0,5738096,90.147873,...,58,0,Bacteria,Bacteroidota,Bacteroidia,Chitinophagales,Chitinophagaceae,Chitinophaga,Chitinophaga pinensis_A,GCA_902167305


### Select a few candidate examples


In [59]:
ochre_phyla = ochre_phage.groupby('phylum').agg('sum').reset_index()
ochre_phyla.sort_values('Kept')

,phylum,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,contig_count,...,trna_count,trna_selenocysteine_count,Genome length,Contigs,Phage Contigs,Total Predicted Prophages,Kept,No phage genes,Not enough genes,bp prophage
5,Desulfobacterota,870,837.67,17.16,2223,1395,240.00,27980313,7.734006e+02,1255,...,371,8,3.267283e+07,957.0,730.0,190.0,8.0,62.0,120.0,2.071120e+05
3,Chloroflexota,111,2346.18,42.55,3952,2692,70.24,112260883,2.193711e+03,11081,...,1161,12,1.286956e+08,9943.0,2810.0,862.0,26.0,391.0,445.0,7.922670e+05
8,Gemmatimonadota,1650,5732.92,166.68,9634,5932,1395.36,196296537,6.261433e+03,16835,...,2337,39,2.103923e+08,14156.0,9150.0,936.0,42.0,546.0,348.0,1.163162e+06
13,Synergistota,1975,3905.17,17.65,4301,2415,296.66,81842508,3.676171e+03,4170,...,1736,13,9.162332e+07,3306.0,1713.0,534.0,54.0,161.0,319.0,1.739537e+06
14,Verrucomicrobiota,3458,9846.03,343.13,26910,17316,3975.34,277225701,1.045643e+04,37096,...,4163,64,3.103284e+08,32445.0,18230.0,4528.0,60.0,3401.0,1067.0,1.177506e+06
10,Nitrospirota,5338,5826.35,196.93,11790,7154,736.06,178792106,5.765020e+03,15123,...,2483,3,2.052842e+08,9957.0,5806.0,1480.0,87.0,557.0,836.0,2.240749e+06
4,Deinococcota,12302,10940.03,150.95,35023,26143,1468.75,293564020,1.014214e+04,19884,...,5309,0,3.235184e+08,19543.0,8605.0,2076.0,140.0,563.0,1373.0,3.809000e+06
11,Planctomycetota,157602,10614.14,230.41,17232,10519,2020.66,579946278,1.070008e+04,29778,...,5355,38,6.631786e+08,25660.0,20452.0,8652.0,307.0,5594.0,2751.0,6.938966e+06
9,Myxococcota,2171,7593.06,175.63,19463,12200,601.23,614570629,7.205088e+03,24144,...,4178,61,6.746475e+08,22704.0,13890.0,7567.0,473.0,4494.0,2600.0,1.033916e+07
7,Firmicutes_A,4826,50833.03,407.04,186549,69248,3315.46,1605801224,4.367840e+04,47276,...,36259,238,1.920572e+09,40838.0,23596.0,13981.0,2122.0,3122.0,8737.0,7.250677e+07


# Acid Mines

This is the email from Sarah Harmer:

>My lab has the following microbes, it would be great to know about their phage:
>
>*Acidithiobacillus ferrooxidans*<br />
>*Leptospirillum ferrooxidans*<br />
>*Acidianus brierleyi* (note: Archaea that is not included in this analysis)
>
>These microbes will be of interest too:<br />
>*Thermoplasmales, Sulfobacillus, Acidimicrobium*, and *Acidiphilium* groups
>
>Anerobic sulfate reducing bacteria are definitely of interest.

See also this table of interesting microbes:
![I bet some of these have phages!](Australia/amd_microbes.png "Do you want to bet some of these have phages?")